<a href="https://colab.research.google.com/github/BaseKan/python_workshop_brh/blob/main/python_broekhuis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>